In [0]:
%sql
CREATE OR REPLACE TABLE workspace.gold_serving.dimension_methods
USING DELTA
AS
SELECT
  time,
  sop,
  method_id,
  meth_id,
  temp_id,
  report_template_version,
  seq_id,
  seq_version
FROM workspace.gold_staging.stg_methods;
